# Make super clean datafile!

respecting the block structure and the fact that the first history columns within a block must be nans. 

In [1]:
import os, sys, glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import operator

# settings for more convenient df inspection
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [2]:
# i need a datafile that also has block type and difficulty level
df_all = pd.read_csv('C:\\Users\\annae\\Desktop\\ChoiceHistory_Psych\\Data\\Exp1_auditory\\experimental_raw\\all_data_blind.csv', delimiter=';')
subjects = list(df_all['Participant Private ID'].dropna().unique())

pdi_res = pd.read_csv('C:\\Users\\annae\\Desktop\\ChoiceHistory_Psych\\Data\\Exp1_auditory\\psychosis_raw\\pdi_privateIDs.csv')
caps_res = pd.read_csv('C:\\Users\\annae\\Desktop\\ChoiceHistory_Psych\\Data\\Exp1_auditory\\psychosis_raw\\caps_privateIDs.csv')

C:\Users\annae\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
len(subjects)

148

In [4]:
# EXCLUSIONS
responses = ['Right', 'Left', '2500']
exclusions, included = [], []

for i in range(1, len(subjects)): 
    # exclude incomplete datasets
    df_sbj = df_all[df_all['Participant Private ID'] == subjects[i]]
    df_sbj_rel = df_sbj[df_sbj['Response'].isin(responses)]
    
    if df_sbj_rel.shape[0] != 385: 
        exclusions.append(subjects[i])
        
    # exclude performance based criterion
    corr = df_sbj_rel['Correct'].value_counts()[1.0]
    wron = df_sbj_rel['Correct'].value_counts()[0.0]
    
    perc = corr / (corr+wron)*100
    
    if perc > 90: 
        exclusions.append(subjects[i])
    elif perc < 60: 
        exclusions.append(subjects[i])
        
    # exclude attention check missers
    attention = df_sbj[df_sbj['Zone Type']=='response_keyboard_single']
    attention_check_correct = attention.Correct.sum()
    
    if attention_check_correct < 20: 
        exclusions.append(subjects[i])
        
    # exclude ppl who didn't make it to the questionnaires
 
    if caps_res[caps_res['Subject_ID']==subjects[i]].shape == (0,7): 
        exclusions.append(subjects[i])
        
    if pdi_res[pdi_res['Subject_ID']==subjects[i]].shape==(0,7):
        exclusions.append(subjects[i])
    
    if subjects[i] not in exclusions: 
        included.append(df_sbj_rel)
        
df_inc = pd.concat(included)
print('Excluded: ',len(set(exclusions)))
print('Included: ',len(included))

Excluded:  34
Included:  113


In [5]:
cols = ['sbj_id','trial','block_type','block','diff','target','response','cue','caps','pdi','motor']

df_reg = pd.DataFrame(columns=cols)


for df_sbj in included: 
    
    block_ends = [49*i for i in range(1,9)]
    trial_ct = 1
    block_ct = 1
    sbj_id = list(df_sbj['Participant Private ID'])[0]
    
    for index, row in df_sbj.iterrows(): 
        
        if trial_ct in block_ends: 
            block_ct += 1
            
        # block_type
        if row['meta_typeblock'] == 'rep': 
            block_type = 0
        elif row['meta_typeblock'] == 'neut': 
            block_type = 1
        
        # cue
        if row['cue'] == 'cue_right.PNG': 
            cue = 0 # right cue
        elif row['cue'] == 'cue_left.PNG': 
            cue = 1 # left cue
        else: 
            print('sth wrong')
            
        # motor response (added for revision)
        if row['ANSWER']==row['up_responsebox']: 
            motor = 0 # upper responsebox selected
        elif row['ANSWER']==row['down_responsebox']: 
            motor = 1 # lower responsebox selected
        
        # stim, target, response
        # 1 (easy), 6 (hard)
        difficulty = row['meta_difflevel'] # in int dtype
        target = row['ANSWER'] # correct answer
        resp = row['Response'] # given answer

        if target == 'Right': 
            target = 0
        elif target == 'Left': 
            target = 1

        if resp == 'Right': 
            resp = 0
        elif resp == 'Left':
            resp = 1
        elif resp == '2500': # time-outs: I classify them as incorrect for now
            
            if target == 1: 
                resp = 0
            elif target == 0: 
                resp = 1
                
        pdi = pdi_res[pdi_res['Subject_ID'] == sbj_id]['sum score'].values[0]
        caps = caps_res[caps_res['Subject_ID'] == sbj_id]['sum score'].values[0]
                
        row = {'sbj_id':sbj_id,'trial':trial_ct,'block_type':block_type,'block':block_ct,
               'diff':difficulty,'target':target,'response':resp,'cue':cue, 'caps':caps, 'pdi':pdi,  
              'motor':motor}
        
        df_reg = df_reg.append(row, ignore_index=True)
        
        trial_ct += 1
        

In [6]:
# for full datafile as used in the incremental model comparison, change to range(1,8)
hists = ['resp_%i'%i for i in range(1,2)]

# make history columns with NaNs
def make_nan_col(df, name): 
    df[name] = np.nan


# collect previous k response and make them NaN at beginning of blocks
def collect_history(df, k):
 
    for index, row in df.iterrows():

        prev = df[df.index==index-k]['response'].values.astype(int)

        if prev.shape == (0,): 
            prev = np.nan
            
        elif df.at[index, 'block'] != df.at[index-1, 'block']: 
            prev = np.nan
            
        else: 
            prev = prev[0]
            
        target_col = str('resp_%i'%k)
        df.at[index, target_col] = prev


In [7]:
for name in hists: 
    make_nan_col(df_reg, name)

# reset index
df = df_reg.reset_index(drop=True)

In [8]:
# for full datafile as used in the incremental model comparison, change to range(1,8)
for k in range(1,2): 
    collect_history(df, k)

In [9]:
def make_nans_history(df, k):
    
    target_col = 'resp_%i'%k
    resps = [0.0, 1.0]
    
    indexes = []
    
    for index, row in df.iterrows():
    
        if df.at[index, target_col] not in resps:
            indexes.append(index)
            
    for i in indexes: 
    
        if i > 7:
            
            for j in range(1,k): 

                df.at[i+j, target_col] = np.nan
              
    return indexes

# add stimulus history

In [10]:
# for full datafile as used in the incremental model comparison, change to range(1,8)
stims = ['stim_%i'%i for i in range(1,2)]

for stim in stims: 
    make_nan_col(df, stim)
    
df = df.reset_index(drop=True)

# collect stimulus history and make it nan at block start
def collect_history(df, k):
 
    for index, row in df.iterrows():

        prev = df[df.index==index-k]['target'].values.astype(int)
        
        if prev.shape == (0,): 
            prev = np.nan
            
        elif df.at[index, 'block'] != df.at[index-1, 'block']: 
            prev = np.nan
            
        else: 
            prev = prev[0]
            
        target_col = str('stim_%i'%k)
        df.at[index, target_col] = prev


In [11]:
# for full datafile as used in the incremental model comparison, change to range(1,8)
for k in range(1,2): 
    collect_history(df, k)

In [12]:
def make_nans_history(df, k):
    
    target_col = 'stim_%i'%k
    stim = [0.0, 1.0]
    
    indexes = []
    
    for index, row in df.iterrows():
    
        if df.at[index, target_col] not in stim:
            indexes.append(index)
            
    for i in indexes: 
    
        if i > 7:
            
            for j in range(1,k): 

                df.at[i+j, target_col] = np.nan
              
    return indexes

In [13]:
# for full datafile as used in the incremental model comparison, change to range(1,8)
for k in range(1,2): 
    make_nans_history(df, k)

In [15]:
# drop random unnamed columns
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# add PPS score, sum of z-standardized pdi and caps scores
pps_measures = ['pdi', 'caps']

for measure in pps_measures: 
    col_zscore = measure + '_zscore'
    df[col_zscore] = (df[measure] - df[measure].mean())/df[measure].std(ddof=0)

# new in revision: z-standardize sum of z-scores
df['PPS'] = df['pdi_zscore'] + df['caps_zscore']

# z-standardise columns
cols = list(df.columns)
[cols.remove(item) for item in ['sbj_id', 'trial', 'block', 'caps_zscore', 'pdi_zscore']]

for col in cols: 
    col_z = col + '_z'
    df[col_z] = (df[col]- df[col].mean())/df[col].std(ddof=0)
    
df = df.dropna()

df.to_csv('1_model1_auditory_REV.csv')

In [16]:
df.head()

,sbj_id,trial,block_type,block,diff,target,response,cue,caps,pdi,motor,resp_1,stim_1,pdi_zscore,caps_zscore,PPS,block_type_z,diff_z,target_z,response_z,cue_z,caps_z,pdi_z,motor_z,resp_1_z,stim_1_z,PPS_z
1,2258445.0,2.0,0.0,1.0,2.0,1.0,1.0,1.0,5.0,5.0,0.0,1.0,1.0,-0.644205,-0.37994,-1.024145,-0.999839,-0.883122,0.997406,1.005324,0.971825,-0.37994,-0.644205,-1.002647,1.002797,0.993006,-0.55441
2,2258445.0,3.0,0.0,1.0,6.0,1.0,0.0,1.0,5.0,5.0,0.0,1.0,1.0,-0.644205,-0.37994,-1.024145,-0.999839,1.457683,0.997406,-0.994704,0.971825,-0.37994,-0.644205,-1.002647,1.002797,0.993006,-0.55441
3,2258445.0,4.0,0.0,1.0,2.0,1.0,1.0,1.0,5.0,5.0,0.0,0.0,1.0,-0.644205,-0.37994,-1.024145,-0.999839,-0.883122,0.997406,1.005324,0.971825,-0.37994,-0.644205,-1.002647,-0.997211,0.993006,-0.55441
4,2258445.0,5.0,0.0,1.0,6.0,1.0,1.0,1.0,5.0,5.0,0.0,1.0,1.0,-0.644205,-0.37994,-1.024145,-0.999839,1.457683,0.997406,1.005324,0.971825,-0.37994,-0.644205,-1.002647,1.002797,0.993006,-0.55441
5,2258445.0,6.0,0.0,1.0,3.0,1.0,1.0,1.0,5.0,5.0,0.0,1.0,1.0,-0.644205,-0.37994,-1.024145,-0.999839,-0.297921,0.997406,1.005324,0.971825,-0.37994,-0.644205,-1.002647,1.002797,0.993006,-0.55441
